In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVR

from sklearn.feature_selection import SequentialFeatureSelector

from Ames_Functions import *
pd.set_option("display.max_columns", None)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
#Loading Datasets and necessary Ames_notebook_state pkl file contents
df_house2021 = load_state_pkl()["df_RE_latlong2021"][0].set_index("SaleID")
df_house2019 = load_state_pkl()["df_RE_latlong2019"][0].set_index("SaleID")
df_houseBoth = df_house2019.append(df_house2021)

df_feature_all2021 = load_state_pkl()["df_house2021_after_feature_engineering"][0] #Dataset ready for regression for 2021
df_feature_all2019 = load_state_pkl()["df_house2019_after_feature_engineering"][0] #Dataset ready for regression for 2019
df_feature_allBoth = load_state_pkl()["df_house_both_after_feature_engineering"][0] #Dataset ready for regression for Both

/Users/lelandmurrin/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/lelandmurrin/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/lelandmurrin/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.0.2 when using version 1.2.

In [4]:
df_feature_allBoth

,Rcrd_Yr,InstOne_Yr,TtlVal_AsrYr,LotArea,YrBuilt,TtlBsmtSF,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,Has_Fireplace,Has_Garage,Has_Bsmt,TotalArea,TotalArea_Gar,AsbShng_Ext,AsphShn_Ext,BrkComm_Ext,BrkFace_Ext,CBlock_Ext,CemntBd_Ext,HdBoard_Ext,ImStucc_Ext,MetalSd_Ext,No_Data_Ext,Other_Ext,Plywood_Ext,Stucco_Ext,VinylSd_Ext,Wd_Sdng_Ext,WdShing_Ext,Two_Types_GrType,Attachd_GrType,Basment_GrType,BuiltIn_GrType,Carport_GrType,Detachd_GrType,No_Data_GrType,None_GrType,One_Story_HsStyle,One_Five_Fin_HsStyle,One_Five_Unf_HsStyle,Two_Story_HsStyle,Two_Five_Fin_HsStyle,Two_Five_Unf_HsStyle,No_Data_HsStyle,S_Foyer_HsStyle,S_Level_HsStyle,Closest_Time,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Gym_closest,Service_High_School_closest,Service_Historic_closest,Service_ISU_closest,Service_Library_closest,Service_Medical_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Closest_5,Convenience_secs,Central_historic_feature,biz_180_secs,biz_480_secs,Year,Stone_Ext
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,2019.0,2019.0,300,8746.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
93,2019.0,2019.0,300,9358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
96,2019.0,2019.0,389500,10015.0,2016.0,1142.0,1.0,0.0,2016.0,3.0,647.0,1,1,1,3385.0,4032.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,243.1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,339.06,487.6,524.0,0,8,2019,0.0
98,2019.0,2019.0,330800,10280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,244.3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,340.26,488.8,525.2,0,8,2019,0.0
100,2021.0,2021.0,399800,10840.0,2020.0,1659.0,0.0,0.0,2020.0,3.0,791.0,0,1,1,3318.0,4109.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,256.8,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,352.76,501.3,537.7,0,8,2019,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,2021.0,2021.0,177600,10776.0,1991.0,950.0,0.0,0.0,2001.0,2.0,747.0,0,1,1,1929.0,2676.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,276.6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,295.28,402.4,391.4,0,15,2021,0.0
22749,2021.0,2021.0,167300,12395.0,1984.0,864.0,0.0,0.0,1984.0,2.0,484.0,0,1,1,1753.0,2237.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,305.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,323.88,431.0,420.0,0,14,2021,0.0
22756,2021.0,2021.0,162900,10463.0,1991.0,962.0,0.0,0.0,1993.0,1.0,384.0,0,1,1,1963.0,2347.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,324.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342.68,449.8,438.8,0,14,2021,0.0


In [10]:
#Taking a look at SalePrice versus Assessor Value of each home for 2019 and 2021

(df_house2019.eval("PriceDiff = abs(SalePrice - TtlVal_AsrYr)").PriceDiff.mean(), 
 df_house2021.eval("PriceDiff = abs(SalePrice - TtlVal_AsrYr)").PriceDiff.mean(),
 df_house2019.eval("PriceRatio = SalePrice/TtlVal_AsrYr").PriceRatio.mean(), 
 df_house2021.eval("PriceRatio = SalePrice/TtlVal_AsrYr").PriceRatio.mean())

#TtlVal_AsrYr field likely updated every time there's an assessment (every odd year) for old records as well. 
#So 2019 records reflect 2021 assessment values that are higher than what they would have been in 2019.

(33587.59600443951, 43168.848341232224, 1.9190268649709301, 1.3348662616524758)

In [12]:
#Checking house sale records where the same home was sold in 2021 and 2019 to see if their recorded assessors values differ or not
df_house2021.merge(df_house2019, on = 'Prop_Addr', suffixes = ["_2021", "_2019"])[["TtlVal_AsrYr_2021", "TtlVal_AsrYr_2019"]].query("TtlVal_AsrYr_2021 == TtlVal_AsrYr_2019")

#Assessor values do not differ, thus the 2019 sale records reflect 2021 assessor values

,TtlVal_AsrYr_2021,TtlVal_AsrYr_2019
0,399800,399800
1,334200,334200
2,313300,313300
3,419300,419300
4,457300,457300
5,389600,389600
6,146200,146200
7,441100,441100
8,331200,331200
9,421000,421000


In [3]:
#NOTE: This cell and the next were looking at a special case where house was flipped essentially (Assessor value and SalePrice for 2019 were very different)

display(df_house2019.eval("PriceRatio = SalePrice/TtlVal_AsrYr").query("PriceRatio == PriceRatio.min()")[["PriceRatio", "TtlVal_AsrYr", "SalePrice", "Prop_Addr", "MoSold_MM", "YrBuilt", "YrSold_YYYY"]])
display(df_house2021[df_house2021.Prop_Addr.str.contains("613 DOTSON")].eval("PriceRatio = SalePrice/TtlVal_AsrYr")[["PriceRatio", "TtlVal_AsrYr", "SalePrice", "Prop_Addr", "MoSold_MM", "YrBuilt", "YrSold_YYYY"]])

,PriceRatio,TtlVal_AsrYr,SalePrice,Prop_Addr,MoSold_MM,YrBuilt,YrSold_YYYY
SaleID,,,,,,,
14581,0.015787,367400,5800.0,613 DOTSON DR,12.0,2020.0,2019.0


,PriceRatio,TtlVal_AsrYr,SalePrice,Prop_Addr,MoSold_MM,YrBuilt,YrSold_YYYY
SaleID,,,,,,,
14580,0.876429,367400,322000.0,613 DOTSON DR,2.0,2020.0,2021.0


# Running preliminary EDA reports on df_feature_all datasets imported from feature engineering notebook

In [ ]:
reportsBoth = EDA_report(df_feature_allBoth.join(df_house2019.SalePrice.append(df_house2021.SalePrice)))
reports2021 = EDA_report(df_feature_all2021.join(df_house2021.SalePrice))
reports2019 = EDA_report(df_feature_all2019.join(df_house2019.SalePrice));

### Displaying feature correlations with Sale Price

In [5]:
(reports2019["corr"]
    .reset_index()
    .join(reports2021["corr"].reset_index(), lsuffix = "_2019", rsuffix = "_2021")
    .join(reportsBoth["corr"].reset_index().add_suffix("_Both"))
    .head(30)
)

,index_2019,SalePrice_2019,index_2021,SalePrice_2021,index_Both,SalePrice_Both
0,SalePrice,1.000000,SalePrice,1.000000,SalePrice,1.000000
1,TtlVal_AsrYr,0.747653,TtlVal_AsrYr,0.827797,TtlVal_AsrYr,0.772447
2,TotalArea_Gar,0.656745,TotalArea_Gar,0.820323,TotalArea_Gar,0.716201
3,TotalArea,0.645802,TotalArea,0.798409,TotalArea,0.701976
4,GarageArea,0.520307,Cars,0.701327,Cars,0.585610
5,Cars,0.519899,TtlBsmtSF,0.693831,TtlBsmtSF,0.578316
6,TtlBsmtSF,0.509157,GarageArea,0.685732,GarageArea,0.577810
7,Has_Fireplace,0.444714,Has_Fireplace,0.547145,Has_Fireplace,0.474991
8,Fireplaces,0.423013,Fireplaces,0.510575,Fireplaces,0.445563
9,Attachd_GrType,0.359747,Attachd_GrType,0.450468,Attachd_GrType,0.383007


### Displaying feature importances from descriptive RFR model with Sale Price as target

In [6]:
(reports2019["important_features"].reset_index(drop = True)
    .join(reports2021["important_features"].reset_index(drop = True), lsuffix = "_2019", rsuffix = "_2021")
    .join(reportsBoth["important_features"].reset_index(drop = True).add_suffix("_Both"))
    .head(30)
)

,Name_2019,Importance_2019,Name_2021,Importance_2021,Name_Both,Importance_Both
0,TtlVal_AsrYr,0.620354,TotalArea_Gar,0.516628,TtlVal_AsrYr,0.527066
1,GarYrBlt,0.081489,TtlVal_AsrYr,0.344485,TotalArea_Gar,0.155886
2,YrBuilt,0.068487,TotalArea,0.031242,TotalArea,0.074176
3,TotalArea_Gar,0.058752,YrBuilt,0.018562,Year,0.052541
4,TotalArea,0.027843,No_Data_Ext,0.011204,YrBuilt,0.034781
5,LotArea,0.020643,Central_historic_feature,0.009813,GarYrBlt,0.025014
6,Rcrd_Yr,0.013531,TtlBsmtSF,0.009236,Closest_Time,0.016735
7,InstOne_Yr,0.01259,LotArea,0.00757,Convenience_secs,0.014814
8,Convenience_secs,0.011442,No_Data_HsStyle,0.007356,LotArea,0.014204
9,Closest_Time,0.011262,Closest_Time,0.007139,Central_historic_feature,0.013298


### Displaying EDA report Descriptive Multiple Linear Regression Summaries

In [7]:
print(reports2021["ols_summary"])

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.823
Method:                 Least Squares   F-statistic:                     48.44
Date:                Fri, 15 Sep 2023   Prob (F-statistic):          2.77e-188
Time:                        17:59:45   Log-Likelihood:                -7693.8
No. Observations:                 633   AIC:                         1.551e+04
Df Residuals:                     570   BIC:                         1.579e+04
Df Model:                          62                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

In [8]:
print(reports2019["ols_summary"]) #.reset_index(drop = True).join(reports2021["important_features"].reset_index(drop = True), lsuffix = "_2019", rsuffix = "_2021").head(30)

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     25.51
Date:                Fri, 15 Sep 2023   Prob (F-statistic):          5.38e-155
Time:                        17:59:46   Log-Likelihood:                -11245.
No. Observations:                 901   AIC:                         2.262e+04
Df Residuals:                     836   BIC:                         2.293e+04
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

In [9]:
print(reportsBoth["ols_summary"])

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     54.48
Date:                Fri, 15 Sep 2023   Prob (F-statistic):               0.00
Time:                        17:59:44   Log-Likelihood:                -19073.
No. Observations:                1534   AIC:                         3.828e+04
Df Residuals:                    1467   BIC:                         3.864e+04
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

# Feature Selection

### LassoCV

In [21]:
#Assigning X and y for lassoCV for 2021 and 2019
X_lassoCV_2019, X_lassoCV_2021, X_lassoCV_Both = df_feature_all2019, df_feature_all2021, df_feature_allBoth
y_lassoCV_2019, y_lassoCV_2021, y_lassoCV_Both = df_house2019.SalePrice, df_house2021.SalePrice, df_houseBoth.SalePrice

In [ ]:
#Fitting lassoCV model to the data and storing selected lasso features for each year
lassoCV_2019, lassoCV_2021, lassoCV_Both = (
                                            LassoCV(normalize = True, max_iter = 10000, random_state = 0), 
                                            LassoCV(normalize = True, max_iter = 10000, random_state = 0), 
                                            LassoCV(normalize = True, max_iter = 10000, random_state = 0)
                                            )

lassoCV_2019.fit(X_lassoCV_2019, y_lassoCV_2019)
lassoCV_2021.fit(X_lassoCV_2021, y_lassoCV_2021)
lassoCV_Both.fit(X_lassoCV_Both, y_lassoCV_Both)

lassoCV_2019_cols, lassoCV_2021_cols, lassoCV_Both_cols = (
                                        lassoCV_2019.feature_names_in_[np.where(lassoCV_2019.coef_ != 0)], 
                                        lassoCV_2021.feature_names_in_[np.where(lassoCV_2021.coef_ != 0)],
                                        lassoCV_Both.feature_names_in_[np.where(lassoCV_Both.coef_ != 0)]
                                        )

In [37]:
#Displaying columns selected by LassoCV
print(lassoCV_2019_cols)
print()
print("-"*80)
print()
print(lassoCV_2021_cols)
print()
print("-"*80)
print()
print(lassoCV_Both_cols)

['Rcrd_Yr' 'InstOne_Yr' 'TtlVal_AsrYr' 'LotArea' 'YrBuilt' 'Fireplaces'
 'PoolArea' 'TotalArea' 'HdBoard_Ext' 'Wd_Sdng_Ext' 'BuiltIn_GrType'
 'Detachd_GrType' 'No_Data_GrType' 'One_Five_Fin_HsStyle'
 'Two_Story_HsStyle' 'No_Data_HsStyle' 'S_Foyer_HsStyle' 'Closest_Time'
 'Service_Gym_closest' 'Service_Historic_closest'
 'Service_Medical_closest' 'Service_Organic_Groceries_closest'
 'Service_Park_closest' 'Service_Religion_closest'
 'Service_Restaurant_closest' 'Central_historic_feature']

--------------------------------------------------------------------------------

['TtlVal_AsrYr' 'YrBuilt' 'Cars' 'GarageArea' 'Has_Fireplace'
 'TotalArea_Gar' 'VinylSd_Ext' 'Detachd_GrType' 'Two_Five_Unf_HsStyle'
 'Service_Restaurant_closest' 'Central_historic_feature' 'biz_180_secs']

--------------------------------------------------------------------------------

['Rcrd_Yr' 'InstOne_Yr' 'TtlVal_AsrYr' 'LotArea' 'YrBuilt' 'Fireplaces'
 'TotalArea' 'TotalArea_Gar' 'BrkFace_Ext' 'HdBoard_Ext' 'Wd_Sd

In [38]:
#running EDA report on selected lassoCV cols

model_EDA2019, model_EDA2021, model_EDABoth = (
                                EDA_report(df_feature_all2019[lassoCV_2019_cols].join(df_house2019.SalePrice)), 
                                EDA_report(df_feature_all2021[lassoCV_2021_cols].join(df_house2021.SalePrice)),
                                EDA_report(df_feature_allBoth[lassoCV_Both_cols].join(df_houseBoth.SalePrice))
                                )


In [40]:
#Checking VIF for lassoCV_Both_cols
pd.DataFrame(np.array([get_vif(df_feature_allBoth[lassoCV_Both_cols]), lassoCV_Both_cols])).T

/Users/lelandmurrin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,0,1
0,inf,Rcrd_Yr
1,inf,InstOne_Yr
2,27.364733,TtlVal_AsrYr
3,3.629288,LotArea
4,315.181906,YrBuilt
5,2.872793,Fireplaces
6,396.38306,TotalArea
7,458.077892,TotalArea_Gar
8,1.517561,BrkFace_Ext
9,1.25223,HdBoard_Ext


# Best Predictive Model Section

## Random Forest Regressor Section

In [46]:
years = {} #master dictionary for storing model results accross multiple regressors, years etc.

years["2019"] = {"df_feature_all": df_feature_all2019, "df_house": df_house2019}
years["2021"] = {"df_feature_all": df_feature_all2021, "df_house": df_house2021}
years["Both"] = {"df_feature_all": df_feature_allBoth, "df_house": df_house2019.append(df_house2021)}

for y in years.values():
    y["RFR"] = {"Score": {}}
    y["RFR"]["Model"] = RandomForestRegressor(random_state = 0)
    y["RFR"]["Model"].set_params(n_estimators = 100, min_samples_split = 3, min_samples_leaf = 1, criterion = 'absolute_error') #params from first round of gridsearchCV on 2021 data for RFR
    y["RFR"]["Model"].fit(y["df_feature_all"], y["df_house"].SalePrice)
    y["RFR"]["Score"]["all"] = y["RFR"]["Model"].score(y["df_feature_all"], y["df_house"].SalePrice)

years["2019"]["RFR"]["Score"], years["2021"]["RFR"]["Score"], years["Both"]["RFR"]["Score"]

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_4282/2380707313.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years["Both"] = {"df_feature_all": df_feature_allBoth, "df_house": df_house2019.append(df_house2021)}


({'all': 0.9290529712587692},
 {'all': 0.9758573695890395},
 {'all': 0.9430390768067685})

In [47]:
#Creating test train splits for 2019, 2021 and Both
for y in years.values():
    y["X"] = {}
    y["y"] = {}
    y["X"]["train"] , y["X"]["test"], y["y"]["train"], y["y"]["test"] = train_test_split(y["df_feature_all"], y["df_house"].SalePrice, test_size = 0.3, random_state = 0)

#X_train, X_test, y_train, y_test = train_test_split(df_feature_all, df_house.SalePrice, test_size = 0.2, random_state = 0)

In [48]:
#Checking test and train scores for 2019, 2021 and Both

for i, y in years.items():
    y["RFR"]["Model"].fit(y["X"]["train"], y["y"]["train"])
    y["RFR"]["Score"]["train"] = y["RFR"]["Model"].score(y["X"]["train"], y["y"]["train"]) 
    y["RFR"]["Score"]["test"] = y["RFR"]["Model"].score(y["X"]["test"], y["y"]["test"]) 
    print(i, "train score:", y["RFR"]["Score"]["train"])
    print(i, "test score:", y["RFR"]["Score"]["test"])

2019 train score: 0.9405152864820496
2019 test score: 0.7072817564836282
2021 train score: 0.9737865422661159
2021 test score: 0.8772843609589269
Both train score: 0.9449543104491841
Both test score: 0.7011891791225657


### RandomizedSearchCV on Random Forest

In [ ]:
params_forest = [{
    "n_estimators": [10, 25, 50, 100, 150],
    "criterion": ['squared_error', 'friedman_mse', 'poisson'],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=10, num=10, dtype=int),
    "random_state": [0]}]

rand_search_RF = RandomizedSearchCV(RandomForestRegressor(), params_forest, scoring = 'r2', cv=5, n_jobs=7, verbose = 3, n_iter = 10)
rand_search_RF.fit(years["Both"]["X"]["train"], years["Both"]["y"]["train"])

In [ ]:
#Manually changed parameters to zero in on best params in preparation for more thorough GridSearchCV

rand_search_RF.best_params_, rand_search_RF.best_score_ 


#2021 SECTION:

# Best params and score WITHOUT ASSESSOR:

#({'random_state': 0,
#   'n_estimators': 100,
#   'min_samples_split': 7,
#   'min_samples_leaf': 1,
#   'criterion': 'squared_error'},
#  0.6352291041964238)

# WITH ASSESSOR:

# Best Params and Score:

# ({'random_state': 0,
#   'n_estimators': 100,
#   'min_samples_split': 3,
#   'min_samples_leaf': 1,
#   'criterion': 'absolute_error'},
#  0.7895917088205711)




# 2019 SECTION:
#BEST PARAMS WITH ASSESSOR:

# ({'random_state': 0,
#   'n_estimators': 150,
#   'min_samples_split': 9,
#   'min_samples_leaf': 3,
#   'criterion': 'absolute_error'},
#  0.7551327849344599)

# train/test scores: (0.8714524101792618, 0.7187271374658213)



#Both Section:

#1st iter:
#({'random_state': 0,
#   'n_estimators': 25,
#   'min_samples_split': 8,
#   'min_samples_leaf': 3,
#   'criterion': 'squared_error'},
#  0.7562236806987308)

#train/test scores:  (0.9147256726117005, 0.8064260689621101)


# 2nd iter:
# ({'random_state': 0,
#   'n_estimators': 25,
#   'min_samples_split': 5,
#   'min_samples_leaf': 3,
#   'criterion': 'squared_error'},
#  0.7536474029903149)

#train/test scores: (0.9218019671548264, 0.8027812415781181)

# 3rd iter: 
# ({'random_state': 0,
#   'n_estimators': 100,
#   'min_samples_split': 3,
#   'min_samples_leaf': 5,
#   'criterion': 'friedman_mse'},
#  0.7660173355211365)

# train/test scores: (0.8965033590134942, 0.7801124903138716)

#Best Params and Score:

# 4th iter:
# ({'random_state': 0,
#   'n_estimators': 50,
#   'min_samples_split': 3,
#   'min_samples_leaf': 3,
#   'criterion': 'squared_error'},
#  0.7634204914173163)

# train/test scores: (0.9235502871706021, 0.8066465892420037)

In [444]:
(
    rand_search_RF.score(years["Both"]["X"]["train"], years["Both"]["y"]["train"]), 
    rand_search_RF.score(years["Both"]["X"]["test"], years["Both"]["y"]["test"])
)

(0.9235502871706021, 0.8066465892420037)

In [ ]:
RFR2 = RandomForestRegressor(**rand_search_RF.best_params_, random_state = 0).fit(years["Both"]["X"]["train"], years["Both"]["y"]["train"])

(
    RFR2.score(years["Both"]["X"]["train"], years["Both"]["y"]["train"]), 
    RFR2.score(years["Both"]["X"]["test"], years["Both"]["y"]["test"])
)

(0.8685949920946678, 0.7841936110014589)

In [220]:
find_important_features(rand_search_RF.best_estimator_)

,Name,Importance
2,TtlVal_AsrYr,0.721491
8,GarYrBlt,0.070024
4,YrBuilt,0.049355
3,LotArea,0.027962
49,Closest_Time,0.012616
...,...,...
20,CBlock_Ext,0.0
7,PoolArea,0.0
18,BrkComm_Ext,0.0
17,AsphShn_Ext,0.0


### GridSearchCV on Random Forest

In [ ]:
params_forest2 = [{
    "n_estimators": [25, 50, 75, 100, 150, 200, 250, 300, 350, 400, 450, 500],
    "criterion": ['squared_error', 'friedman_mse', 'poisson'],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=14, num=13, dtype=int),
    "random_state": [0]}]

grid_search_RF = GridSearchCV(RandomForestRegressor(), params_forest2, scoring = 'r2', cv=5, n_jobs=7, verbose = 3)
grid_search_RF.fit(years["Both"]["X"]["train"], years["Both"]["y"]["train"])

# BEST PARAMS:
# ({'criterion': 'squared_error',
#   'min_samples_leaf': 5,
#   'min_samples_split': 2,
#   'n_estimators': 50,
#   'random_state': 0},
#  0.7685783438019305)


# ({'criterion': 'squared_error',
#   'min_samples_leaf': 4,
#   'min_samples_split': 10,
#   'n_estimators': 75,
#   'random_state': 0},
#  0.7698674662071674)

# (0.9037962366330242, 0.7847113976157467)

In [469]:
grid_search_RF.best_params_, grid_search_RF.best_score_

({'criterion': 'squared_error',
  'min_samples_leaf': 4,
  'min_samples_split': 10,
  'n_estimators': 75,
  'random_state': 0},
 0.7698674662071674)

In [470]:
(
    grid_search_RF.score(years["Both"]["X"]["train"], years["Both"]["y"]["train"]), 
    grid_search_RF.score(years["Both"]["X"]["test"], years["Both"]["y"]["test"])
)

(0.9037962366330242, 0.7847113976157467)

## Support Vector Regressor Section

In [94]:
SS = StandardScaler().fit(years["Both"]["df_feature_all"])
years["Both"]["X"]["trainSS"] = SS.transform(years["Both"]["X"]["train"])
years["Both"]["X"]["testSS"] = SS.transform(years["Both"]["X"]["test"]) 
years["Both"]["y"]["trainSS"] = years["Both"]["y"]["train"]
years["Both"]["y"]["testSS"] = years["Both"]["y"]["test"]

In [95]:
SVR_model = SVR(epsilon = 1e-4, gamma=1)

for t in ["trainSS", "testSS"]:
    print(t)
    print(SVR_model
          .fit(years["Both"]["X"]["trainSS"], years["Both"]["y"]["trainSS"])
          .score(years["Both"]["X"][t], years["Both"]["y"][t])
          )

trainSS
-0.019636365068382178
testSS
-0.00225665639976258


### RandomizedSearchCV on SVR

In [ ]:
best_params = []
best_score = []

params = {
    'kernel' : ['poly', 'linear', 'rbf'],
    'C' : np.linspace(1e-16, 1e6, 20),
    'gamma' : np.linspace(1e-10, 1e4, 20),
    'epsilon': np.linspace(1e-10, 10, 20)
}

    rand_search_SVR = RandomizedSearchCV(SVR_model, params, cv = 3, return_train_score = True, n_jobs = 7, verbose = 3, n_iter = 5)
    rand_search_SVR.fit(years["Both"]["X"]["trainSS"], years["Both"]["y"]["trainSS"])

In [316]:
rand_search_SVR.best_params, rand_search_SVR.best_score #best we got with X_trainSS params: ([{'epsilon': 8.947369473684211e-05, 'C': 221315789.47368422}], score: [0.6512524018396496])


#1st iter:

# ([{'kernel': 'linear',
#    'epsilon': 2.6315789474421054,
#    'C': 105263.15789563156}],
#  [0.4346098118608208])

# 2nd iter:
# ([{'kernel': 'linear',
#    'epsilon': 2.1052631579736842,
#    'C': 52631.578948315786}],
#  [0.4389655032282274])

# 3rd iter:
# ([{'kernel': 'linear',
#    'epsilon': 5.7894736842526315,
#    'C': 157894.73684294737}],
#  [0.432610572283052])

# 4th iter (INCLUDES GAMMA):
# ([{'kernel': 'linear',
#    'gamma': 8421.052631578963,
#    'epsilon': 2.1052631579736842,
#    'C': 105263.15789473684}],
#  [0.4346047467011582])

([{'kernel': 'linear',
   'gamma': 8421.052631578963,
   'epsilon': 2.1052631579736842,
   'C': 105263.15789473684}],
 [0.4346047467011582])

### GridSearchCV on SVR

In [ ]:
params = {
    'kernel' : ['rbf', 'linear', 'poly'],
    'C' : np.linspace(1e-4, 1e7, 20),
    'gamma' : np.linspace(1e-15, 1e4, 20),
    'epsilon': np.linspace(1e-8, 1e6, 20)
}

grid_search_SVR = GridSearchCV(SVR_model, params, cv = 10, return_train_score = True, n_jobs = -1, verbose = 3)
grid_search_SVR.fit(years["Both"]["X"]["trainSS"], years["Both"]["y"]["trainSS"])

In [ ]:
grid_search_SVR.best_params, grid_search_SVR.best_score

## Gradient Boosting Regressor Section

In [320]:
GBR = GradientBoostingRegressor()

In [475]:
for y in years.values():
    y["GBR"] = {"Score": {}}
    y["GBR"]["Model"] = GradientBoostingRegressor(random_state = 0)
    y["GBR"]["Model"].fit(y["df_feature_all"], y["df_house"].SalePrice)
    y["GBR"]["Score"]["all"] = y["GBR"]["Model"].score(y["df_feature_all"], y["df_house"].SalePrice)

years["2019"]["GBR"]["Score"], years["2021"]["GBR"]["Score"], years["Both"]["GBR"]["Score"]

({'all': 0.9180112633222844},
 {'all': 0.9673950832252093},
 {'all': 0.9114388874089325})

In [476]:
#Fitting model and producing initial train and test scores
GBR.fit(years["Both"]["X"]["train"], years["Both"]["y"]["train"])
GBR.score(years["Both"]["X"]["train"], years["Both"]["y"]["train"]), GBR.score(years["Both"]["X"]["test"], years["Both"]["y"]["test"])

(0.9269280355706639, 0.7783670607499308)

### GridSearchCV on GradientBoosting

In [ ]:
params = {
    #'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'criterion': ['friedman_mse', 'squared_error'],
    'learning_rate': [0.065, 0.07, 0.075],
    'max_depth': [1, 2, 3],
    'min_samples_split': [4, 6, 8],
    'n_estimators': [100, 105, 110],
    'subsample': [0.75, 0.8, 0.85]
}

grid_search_GBR = GridSearchCV(estimator=GradientBoostingRegressor(random_state = 0), param_grid=params, cv=3, n_jobs=7, verbose = 3)
grid_search_GBR.fit(years["Both"]["X"]["train"], years["Both"]["y"]["train"])

#Both:
# 1st iter:
# ({'criterion': 'squared_error',
#   'learning_rate': 0.1,
#   'max_depth': 2,
#   'min_samples_split': 2,
#   'n_estimators': 100,
#   'subsample': 0.75},
#  0.7667796960048786)

# 2nd iter: 
# ({'criterion': 'friedman_mse',
#   'learning_rate': 0.075,
#   'max_depth': 2,
#   'min_samples_split': 6,
#   'n_estimators': 105,
#   'subsample': 0.8},
#  0.7707535673022793)

#2019 alone:

#BEST PARAMS FOUND ON 8TH ITER:
# 8th:
# ({'criterion': 'friedman_mse',
#   'learning_rate': 0.12,
#   'max_depth': 3,
#   'min_samples_split': 6,
#   'n_estimators': 35,
#   'subsample': 0.8},
#  0.7668136594192054)


In [500]:
grid_search_GBR.best_params_, grid_search_GBR.best_score_

({'criterion': 'friedman_mse',
  'learning_rate': 0.075,
  'max_depth': 2,
  'min_samples_split': 6,
  'n_estimators': 105,
  'subsample': 0.8},
 0.7707535673022793)

In [504]:
find_important_features(grid_search_GBR.best_estimator_).head(20)

,Name,Importance
2,TtlVal_AsrYr,0.48371
15,TotalArea_Gar,0.238875
14,TotalArea,0.086339
71,Year,0.057185
4,YrBuilt,0.041726
8,GarYrBlt,0.033717
5,TtlBsmtSF,0.018949
49,Closest_Time,0.013551
68,Central_historic_feature,0.008181
46,No_Data_HsStyle,0.004428


In [501]:
(
grid_search_GBR.score(years["Both"]["X"]["train"], years["Both"]["y"]["train"]), 
grid_search_GBR.score(years["Both"]["X"]["test"], years["Both"]["y"]["test"])
)

(0.8685949920946678, 0.7841936110014589)

In [502]:
GBR2 = GradientBoostingRegressor(**grid_search_GBR.best_params_, random_state = 0).fit(years["Both"]["X"]["train"], years["Both"]["y"]["train"])

(
GBR2.score(years["Both"]["X"]["train"], years["Both"]["y"]["train"]), 
GBR2.score(years["Both"]["X"]["test"], years["Both"]["y"]["test"])
)

(0.8685949920946678, 0.7841936110014589)

## LabelEncoder on Categorical Variables Section to add additional fields

In [56]:
all_cat_house_cols = years["Both"]["df_house"].dropna(axis = 1).select_dtypes(include = 'object').columns.tolist()
all_cat_house_cols

['Prop_Addr',
 'ZngCdPr',
 'ClassPr_S',
 'ClassSc_S',
 'Legal_Pr',
 'MA_Ownr1',
 'Inst1_No',
 'Inst1TPr',
 'ValType',
 'Neighborhood',
 'ParType',
 'BldgType',
 'HouseStyle',
 'Foundation',
 'RoofMatl',
 'Ext1',
 'Ext2',
 'MasVnrType',
 'Heating',
 'Central Air',
 'GarageType',
 'SaleType',
 'SaleCond',
 'ParclRel',
 'PA-Nmbr',
 'PA-Strt',
 'Date',
 'Source',
 'reverse_lookup_rd']

In [51]:
#Selecting categorical fields in preparation for label encoding
selected_cat_house_cols = ['ZngCdPr', 'ClassSc_S', 'Inst1TPr', 'ValType', 'Neighborhood',
'ParType', 'BldgType', 'HouseStyle', 'Foundation', 'RoofMatl', 'Ext1', 'Ext2',
'MasVnrType', 'Heating', 'Central Air', 'GarageType', 'SaleType', 'SaleCond',
'ParclRel']

In [55]:
#Encoding selected categorical fields and combining them with numerical fields of interest
LE = LabelEncoder()

unselected_num_house_cols = ['SalePrice', 'MapRefNo', 'GeoRefNo', 'Tier', 
'Range', 'ImpAc_S', 'ImpAcX2S', 'HSTtl_D', 'MilVal_D', 
'HSTtl_S', 'MilVal_S', 'AcreX_S1', 'AcreNt_S', 'BldgNo_S']

df_features_house = (years["Both"]["df_house"]
    .drop(all_cat_house_cols + unselected_num_house_cols, axis = 1)
    .dropna(axis = 1)
    .join(years["Both"]["df_house"][selected_cat_house_cols]
            .apply(lambda x: LE.fit_transform(x)))
)
df_features_house

,SchD_S,TxD_S,Rcrd_Yr,Rcrd_Mo,Inst1_Yr,Inst1_Mo,LndAc_S,OthAc_S,TtlVal_AsrYr,LndAcX1S,ImpAcX1S,AcreGr,LotArea,DwlgNo_S,YrBuilt,GLA,TtlBsmtSF,TotRmsAbvGrd,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,YrSold_YYYY,MoSold_MM,NmbrBRs,Lat,Long,ZngCdPr,ClassSc_S,Inst1TPr,ValType,Neighborhood,ParType,BldgType,HouseStyle,Foundation,RoofMatl,Ext1,Ext2,MasVnrType,Heating,Central Air,GarageType,SaleType,SaleCond,ParclRel
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,5,45,2019.0,11.0,2019.0,11.0,300,0,300,0,0,0.0,8746.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,10.0,0.0,42.074875,-93.635298,3,21,13,3,11,2,4,6,2,3,7,9,2,5,1,6,6,4,1
93,5,45,2019.0,11.0,2019.0,11.0,300,0,300,0,0,0.0,9358.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,10.0,0.0,42.074875,-93.635298,3,21,13,3,11,2,4,6,2,3,7,9,2,5,1,6,6,4,1
96,5,45,2019.0,11.0,2019.0,11.0,97400,292100,389500,0,0,0.0,10015.0,1,2016.0,2243.0,1142.0,9.0,1.0,0.0,2016.0,3.0,647.0,2019.0,11.0,4.0,42.077416,-93.629924,3,12,13,1,11,1,0,3,1,0,10,14,1,0,2,1,6,4,1
98,5,45,2019.0,11.0,2019.0,11.0,82700,248100,330800,0,0,0.0,10280.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,10.0,0.0,42.077403,-93.630016,3,6,12,1,11,2,4,6,2,3,7,9,2,5,1,6,6,2,1
100,5,45,2021.0,6.0,2021.0,6.0,100000,299800,399800,0,0,0.0,10840.0,1,2020.0,1659.0,1659.0,7.0,0.0,0.0,2020.0,3.0,791.0,2019.0,10.0,3.0,42.077346,-93.631034,3,6,13,1,11,1,0,0,3,0,10,14,4,0,2,1,6,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,1,1,2021.0,1.0,2021.0,1.0,44400,133200,177600,0,0,0.0,10776.0,1,1991.0,979.0,950.0,6.0,0.0,0.0,2001.0,2.0,747.0,2021.0,1.0,3.0,41.986503,-93.606026,9,19,13,1,16,1,0,8,3,0,5,16,3,0,0,5,6,4,1
22749,1,1,2021.0,6.0,2021.0,6.0,41800,125500,167300,0,0,0.0,12395.0,1,1984.0,889.0,864.0,6.0,0.0,0.0,1984.0,2.0,484.0,2021.0,6.0,3.0,41.986496,-93.603619,9,6,13,1,16,1,0,0,1,0,5,11,3,0,2,1,6,4,1
22756,1,1,2021.0,5.0,2021.0,5.0,40700,122200,162900,0,0,0.0,10463.0,1,1991.0,1001.0,962.0,6.0,0.0,0.0,1993.0,1.0,384.0,2021.0,5.0,3.0,41.986499,-93.602049,9,19,12,1,16,1,0,8,3,0,5,16,3,0,2,5,6,4,1


In [59]:
#df_features_house.rename(columns = {col: col.replace(" ", "_").replace("1", "One").replace("2", "Two").replace("5", "Five") for col in df_features_house.columns}, inplace = True)
results_EDA_house = EDA_report(df_features_house.join(years["Both"]["df_house"].SalePrice))

In [60]:
print(results_EDA_house['ols_summary'])

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.757
Method:                 Least Squares   F-statistic:                     112.1
Date:                Sat, 16 Sep 2023   Prob (F-statistic):               0.00
Time:                        12:56:45   Log-Likelihood:                -18916.
No. Observations:                1534   AIC:                         3.792e+04
Df Residuals:                    1490   BIC:                         3.816e+04
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -1.728e+07   4.55e+06     -3.795   

In [61]:
results_EDA_house["important_features"]

,Name,Importance
7,OthAc_S,0.347092
8,TtlVal_AsrYr,0.272409
45,SaleCond,0.10381
6,LndAc_S,0.054545
27,Long,0.022329
14,YrBuilt,0.021205
15,GLA,0.015954
23,YrSold_YYYY,0.015879
20,GarYrBlt,0.015185
16,TtlBsmtSF,0.012692


In [62]:
RFR_house = RandomForestRegressor(random_state = 0)

data_train, data_test = train_test_split(df_features_house.join(years["Both"]["df_house"].SalePrice), test_size = 0.3, random_state = 0)

In [63]:
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = data_train.drop("SalePrice", axis = 1), data_test.drop("SalePrice", axis = 1), data_train.SalePrice, data_test.SalePrice

In [64]:
RFR_house.fit(X_train_encoded, y_train_encoded)
RFR_house.score(X_train_encoded, y_train_encoded), RFR_house.score(X_test_encoded, y_test_encoded)

(0.9705143067545015, 0.7364344474155085)

In [65]:
#Creating final df_features_all with encoded fields
df_features_all_encoded = df_features_house.join(years["Both"]["df_feature_all"].iloc[:, 11:].filter(regex = "Has|Area|Closest|closest|secs|historic|biz|Year"))

In [ ]:
#Creating train_test_split for df_features_all_encoded and instantiating RFR_enc for preliminary evaluation
RFR_enc = RandomForestRegressor(random_state = 0)

years["Both"]["X"]["train_enc"], years["Both"]["X"]["test_enc"], years["Both"]["y"]["train_enc"], years["Both"]["y"]["test_enc"] = (train_test_split(df_features_all_encoded, 
                                                                                                                                            years["Both"]["df_house"].SalePrice, 
                                                                                                                                            test_size = 0.3, 
                                                                                                                                            random_state = 0)
                                                                                                                                    )


In [111]:
RFR_enc.fit(years["Both"]["X"]["train_enc"], years["Both"]["y"]["train_enc"])
RFR_enc.score(years["Both"]["X"]["train_enc"], years["Both"]["y"]["train_enc"]), RFR_enc.score(years["Both"]["X"]["test_enc"], years["Both"]["y"]["test_enc"])

(0.9708396924478238, 0.7630281307046058)

In [112]:
find_important_features(RFR_enc).head(20)

,Name,Importance
51,TotalArea_Gar,0.222217
7,OthAc_S,0.21916
8,TtlVal_AsrYr,0.192483
45,SaleCond,0.077197
6,LndAc_S,0.066106
50,TotalArea,0.050904
26,Lat,0.015074
52,Closest_Time,0.011908
31,ValType,0.010887
14,YrBuilt,0.008526


In [71]:
results_EDA_enc = EDA_report(df_features_all_encoded.join(years["Both"]["df_house"].SalePrice))

In [114]:
results_EDA_enc["corr"].head(20)

SalePrice                   1.000000
TtlVal_AsrYr                0.772447
OthAc_S                     0.771603
TotalArea_Gar               0.716201
LndAc_S                     0.713472
TotalArea                   0.701976
GLA                         0.634966
Cars                        0.585610
TtlBsmtSF                   0.578316
GarageArea                  0.577810
Has_Fireplace               0.474991
TotRmsAbvGrd                0.458825
Fireplaces                  0.445563
SaleCond                    0.350505
Central_historic_feature    0.314449
TxD_S                       0.307561
SchD_S                      0.307561
LotArea                     0.301899
GarYrBlt                    0.300668
Has_Garage                  0.283375
Name: SalePrice, dtype: float64

## GridSearchCV on RandomForest for df_Feature_all_encoded

In [ ]:
params_forest3 = [{
    "n_estimators": [35, 45, 55],
    "criterion": ['squared_error', 'friedman_mse'],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=10, num=9, dtype=int),
    "random_state": [0]}]

grid_search_RF2 = GridSearchCV(RandomForestRegressor(), params_forest3, scoring = 'r2', cv=3, n_jobs=7, verbose = 3)
grid_search_RF2.fit(years["Both"]["X"]["train_enc"], years["Both"]["y"]["train_enc"])

#1st iter:
#({'criterion': 'squared_error',
#   'min_samples_leaf': 2,
#   'min_samples_split': 5,
#   'n_estimators': 50,
#   'random_state': 0},
#  0.7783955244400095)

#2nd iter:
# ({'criterion': 'squared_error',
#   'min_samples_leaf': 3,
#   'min_samples_split': 7,
#   'n_estimators': 50,
#   'random_state': 0},
#  0.7697003230160302)

#3rd iter:
# ({'criterion': 'squared_error',
#   'min_samples_leaf': 3,
#   'min_samples_split': 7,
#   'n_estimators': 45,
#   'random_state': 0},
#  0.7698818213285632)

In [723]:
grid_search_RF2.best_params_, grid_search_RF2.best_score_

({'criterion': 'squared_error',
  'min_samples_leaf': 2,
  'min_samples_split': 5,
  'n_estimators': 35,
  'random_state': 0},
 0.7515010243474762)

In [727]:
find_important_features(grid_search_RF2.best_estimator_).head(20)

,Name,Importance
8,TtlVal_AsrYr,0.233333
7,OthAc_S,0.212948
45,SaleCond,0.140681
51,TotalArea_Gar,0.081266
50,TotalArea,0.081012
6,LndAc_S,0.046099
20,GarYrBlt,0.019571
14,YrBuilt,0.015564
12,LotArea,0.015387
26,Lat,0.015111


In [726]:
(
grid_search_RF2.score(years["Both"]["X"]["train_enc"], years["Both"]["y"]["train_enc"]), 
grid_search_RF2.score(years["Both"]["X"]["test_enc"], years["Both"]["y"]["test_enc"])
)

(0.9384176398848866, 0.7795892473694355)

In [725]:
RFR2 = RandomForestRegressor(**grid_search_RF2.best_params_)

RFR2.fit(years["Both"]["X"]["train_enc"], years["Both"]["y"]["train_enc"])

(
RFR2.score(years["Both"]["X"]["train_enc"], years["Both"]["y"]["train_enc"]), 
RFR2.score(years["Both"]["X"]["test_enc"], years["Both"]["y"]["test_enc"])
)

(0.9465214402387343, 0.7749020669048002)

# Feature Selection on df_feature_all_encoded

## SequentialFeatureSelector on RandomForest section

In [159]:
#Instantiating the greedy algorithm with forward direction
SFS = SequentialFeatureSelector(RandomForestRegressor(random_state = 0), direction = "forward", n_jobs = 7)

In [116]:
#This cell takes a while to run (~14mins)

SFS.fit(years["Both"]["X"]["train_enc"], years["Both"]["y"]["train_enc"])

In [185]:
#Storing selected features from SFS
greedy_selected_features = SFS.get_feature_names_out()
greedy_selected_features

array(['Rcrd_Yr', 'OthAc_S', 'LndAcXOneS', 'AcreGr', 'LotArea', 'YrBuilt',
       'GarYrBlt', 'Cars', 'YrSold_YYYY', 'MoSold_MM', 'InstOneTPr',
       'BldgType', 'HouseStyle', 'Foundation', 'RoofMatl', 'ExtOne',
       'ExtTwo', 'Heating', 'Central_Air', 'GarageType', 'SaleType',
       'SaleCond', 'ParclRel', 'Has_Fireplace', 'Has_Garage',
       'TotalArea_Gar', 'Service_Arts_closest',
       'Service_Elem_School_closest', 'Service_Golf_closest',
       'Service_Organic_Groceries_closest', 'Service_Park_closest',
       'Service_Recreation_closest', 'Service_Religion_closest',
       'Service_Restaurant_closest', 'Service_Shopping_closest',
       'Service_Spa_closest', 'Year'], dtype=object)

In [126]:
#Storing train and test X data for RFR_SFS selected features
years["Both"]["X"]["train_RFR_SFS"], years["Both"]["X"]["test_RFR_SFS"] = years["Both"]["X"]["train_enc"][greedy_selected_features], years["Both"]["X"]["test_enc"][greedy_selected_features]

In [194]:
#Preliminary check on SFS dataset performance with RFR model
RFR_SFS = RandomForestRegressor(random_state = 0)
RFR_SFS.fit(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]['train_enc'])
RFR_SFS.score(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]['train_enc']), RFR_SFS.score(years["Both"]["X"]["test_RFR_SFS"], years["Both"]["y"]['test_enc'])

(0.9760192485470717, 0.7987792361819216)

### GridSearchCV on RFR with SFS selected features

In [ ]:
params_RFR_SFS = [{
    "n_estimators": [40, 45, 50],
    "criterion": ['squared_error', 'friedman_mse'],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=10, num=9, dtype=int),
    "random_state": [0]}]

grid_search_RFR_SFS = GridSearchCV(RandomForestRegressor(), params_RFR_SFS, scoring = 'r2', cv=3, n_jobs=7, verbose = 3)
grid_search_RFR_SFS.fit(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]['train_enc'])

# #({'criterion': 'squared_error',
#   'min_samples_leaf': 3,
#   'min_samples_split': 2,
#   'n_estimators': 45,
#   'random_state': 0},
#  0.7962102644110419)

In [764]:
grid_search_RFR_SFS.best_params_, grid_search_RFR_SFS.best_score_

({'criterion': 'squared_error',
  'min_samples_leaf': 3,
  'min_samples_split': 2,
  'n_estimators': 50,
  'random_state': 0},
 0.7969190518707143)

In [765]:
(
grid_search_RFR_SFS.score(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]['train_enc']), 
grid_search_RFR_SFS.score(years["Both"]["X"]["test_RFR_SFS"], years["Both"]["y"]['test_enc'])
)

(0.9377064599460966, 0.7985731663852618)

## Gradient Boosting on SFS selected features dataset

### Testing SFS selected features datasets on GBR

In [199]:
GBR_SFS = GradientBoostingRegressor(random_state = 0)
GBR_SFS.fit(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]["train_enc"])
GBR_SFS.score(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]["train_enc"]), GBR_SFS.score(years["Both"]["X"]["test_RFR_SFS"], years["Both"]["y"]["test_enc"])

(0.9543393873811429, 0.8697830638507938)

In [250]:
params_GBR_SFS= {
    #'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'criterion': ['friedman_mse', 'squared_error'],
    'learning_rate': [0.068, 0.07, 0.072],
    'max_depth': [3, 4, 5],
    'min_samples_split': [6, 7, 8],
    'n_estimators': [200, 210, 220],
    'subsample': [0.68, 0.70, 0.72]
}

grid_search_GBR_SFS = GridSearchCV(estimator=GradientBoostingRegressor(random_state = 0), param_grid=params_GBR_SFS, cv=3, n_jobs=7, verbose = 3)
grid_search_GBR_SFS.fit(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]["train_enc"])

# 1st iter:
# #({'criterion': 'squared_error',
#   'learning_rate': 0.075,
#   'max_depth': 3,
#   'min_samples_split': 6,
#   'n_estimators': 110,
#   'subsample': 0.75},
#  0.8478281570058347)

#2nd iter:
# ({'criterion': 'squared_error',
#   'learning_rate': 0.07,
#   'max_depth': 4,
#   'min_samples_split': 6,
#   'n_estimators': 120,
#   'subsample': 0.75},
#  0.854467084912816)

#5th iter:
# ({'criterion': 'friedman_mse',
#   'learning_rate': 0.07,
#   'max_depth': 4,
#   'min_samples_split': 7,
#   'n_estimators': 200,
#   'subsample': 0.7},
#  0.8621866763551239)

#6th iter:
# ({'criterion': 'friedman_mse',
#   'learning_rate': 0.07,
#   'max_depth': 4,
#   'min_samples_split': 7,
#   'n_estimators': 200,
#   'subsample': 0.7},
#  0.8621866763551239)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits
[CV 1/3] END criterion=friedman_mse, learning_rate=0.068, max_depth=3, min_samples_split=6, n_estimators=200, subsample=0.7;, score=0.880 total time=   0.2s
[CV 3/3] END criterion=friedman_mse, learning_rate=0.068, max_depth=3, min_samples_split=6, n_estimators=200, subsample=0.68;, score=0.822 total time=   0.3s
[CV 2/3] END criterion=friedman_mse, learning_rate=0.068, max_depth=3, min_samples_split=6, n_estimators=200, subsample=0.68;, score=0.840 total time=   0.2s
[CV 1/3] END criterion=friedman_mse, learning_rate=0.068, max_depth=3, min_samples_split=6, n_estimators=200, subsample=0.68;, score=0.865 total time=   0.3s
[CV 2/3] END criterion=friedman_mse, learning_rate=0.068, max_depth=3, min_samples_split=6, n_estimators=200, subsample=0.7;, score=0.843 total time=   0.2s
[CV 1/3] END criterion=friedman_mse, learning_rate=0.068, max_depth=3, min_samples_split=6, n_estimators=200, subsample=0.72;, score=0.864 total tim

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=0),
             n_jobs=7,
             param_grid={'criterion': ['friedman_mse', 'squared_error'],
                         'learning_rate': [0.068, 0.07, 0.072],
                         'max_depth': [3, 4, 5], 'min_samples_split': [6, 7, 8],
                         'n_estimators': [200, 210, 220],
                         'subsample': [0.68, 0.7, 0.72]},
             verbose=3)

In [251]:
grid_search_GBR_SFS.best_params_, grid_search_GBR_SFS.best_score_

({'criterion': 'friedman_mse',
  'learning_rate': 0.07,
  'max_depth': 4,
  'min_samples_split': 7,
  'n_estimators': 200,
  'subsample': 0.7},
 0.8621866763551239)

In [252]:
(
grid_search_GBR_SFS.score(years["Both"]["X"]["train_RFR_SFS"], years["Both"]["y"]["train_enc"]), 
grid_search_GBR_SFS.score(years["Both"]["X"]["test_RFR_SFS"], years["Both"]["y"]["test_enc"])
)

(0.9819143545199268, 0.8864105763412109)

## Storing Final Model with datasets and GridSearchCV and outputting to Ames_notebook_state pkl file

In [890]:
years_final = {}
years_final["df_features_all_encoded"] = df_features_all_encoded
years_final["X"] = {
                    "train" : years["Both"]['X']['train_enc'], 
                    "test" : years["Both"]['X']['test_enc'], 
                    "train_SFS": years["Both"]['X']['train_RFR_SFS'], 
                    "test_SFS": years["Both"]['X']['test_RFR_SFS']
                    }
years_final["y"] = {
                    "train" : years["Both"]['y']['train_enc'], 
                    "test" : years["Both"]['y']['test_enc']
                    }
years_final["SFS_selected_features"] = greedy_selected_features
years_final["GBR"] = {
                        "Model": grid_search_GBR_SFS.best_estimator_, 
                        "Score": {}
                    }
years_final["GBR"]["Score"] = {
                                "train": grid_search_GBR_SFS.best_estimator_.score(years["Both"]['X']['train_RFR_SFS'], years["Both"]['y']['train']), 
                                "test": grid_search_GBR_SFS.best_estimator_.score(years["Both"]['X']['test_RFR_SFS'],years["Both"]['y']['test'])
                                }
years_final["GBR"]["GridSearchCV"] = grid_search_GBR_SFS
years_final["df_features_all_before_encoding"] = years["Both"]["df_house"][selected_cat_house_cols]

In [895]:
save_state_pkl("Final_Model",
                years_final,
                "Contains best GBR model produced along with scores/GridSearchCV, train_test split data after encoding, df_features_all plus encoded features, selected features from SFS, and the dataset containing fields prior to encoding")

# Making Predictions on a Generated Test House Section

In [260]:
#Finding houses to use for sample house sale presentation section 
(df_houseBoth
            .loc[load_state_pkl()["Final_Model"][0]["X"]['test_SFS']
                .query("Has_Fireplace == 0 and Has_Garage == 0 and Service_Organic_Groceries_closest == 0 and Service_Spa_closest == 0 and Year == 2019")
                .index]
            .pipe(remove_undeveloped_lots)
            .BldgType
)

SaleID
20417      Condo
22499    Twnhs-E
13092      Condo
22472    Twnhs-I
13077      Condo
12480      Condo
20396      Condo
4212       Condo
6195     1FamDet
9482     2FmConv
8151     1FamDet
10434    2FmConv
Name: BldgType, dtype: object

In [261]:
#Synthesize records based on selected houses and calculating y_predict based on final model when adding in features 
attr_dict = {}
attr_dict["Has_Fireplace"] = 1
attr_dict["Has_Garage"] = 1
attr_dict["Service_Organic_Groceries_closest"] = 1
attr_dict["Service_Restaurant_closest"] = 1
attr_dict["Service_Shopping_closest"] = 1
attr_dict["Service_Religion_closest"] = 1
attr_dict["Year"] = 2021
yy = load_state_pkl()["Final_Model"][0]["y"]["test"]
XX = load_state_pkl()["Final_Model"][0]["X"]["test_SFS"]
df_predict_list = []

for SaleID in [8151, 22472, 13077, 6195, 20417, 4212, 9482, 10434, 14799, 10653, 14817, 14787]:
    row = load_state_pkl()["Final_Model"][0]["X"]['test_SFS'].query("SaleID == @SaleID")
    X_predict = pd.DataFrame([row.copy().eval(f"{k} = {v}").T[SaleID] for k,v in attr_dict.items()])
    y_true = yy[SaleID]
    y_predict = load_state_pkl()["Final_Model"][0]["GBR"]["Model"].predict(X_predict)
    y_delta = y_predict - y_true
    df_predict_list.append(X_predict.eval("y_predict = @y_predict").eval("y_true = @y_true").eval("y_delta = @y_delta"))

df_predict = pd.concat(df_predict_list).rename_axis("SaleID").reset_index()
    

df_predict.sort_values("y_true", ascending = False)#.query("SaleID == 10434")

,SaleID,Rcrd_Yr,OthAc_S,LndAcXOneS,AcreGr,LotArea,YrBuilt,GarYrBlt,Cars,YrSold_YYYY,MoSold_MM,InstOneTPr,BldgType,HouseStyle,Foundation,RoofMatl,ExtOne,ExtTwo,Heating,Central_Air,GarageType,SaleType,SaleCond,ParclRel,Has_Fireplace,Has_Garage,TotalArea_Gar,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Year,y_predict,y_true,y_delta
49,10434,2019.0,123800.0,0.0,0.0,9600.0,1930.0,0.0,0.0,2019.0,7.0,12.0,1.0,0.0,1.0,0.0,11.0,15.0,0.0,2.0,7.0,6.0,4.0,1.0,1.0,0.0,2296.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,189684.514427,205000.0,-15315.485573
55,10434,2019.0,123800.0,0.0,0.0,9600.0,1930.0,0.0,0.0,2019.0,7.0,12.0,1.0,0.0,1.0,0.0,11.0,15.0,0.0,2.0,7.0,6.0,4.0,1.0,0.0,0.0,2296.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021.0,194584.337818,205000.0,-10415.662182
54,10434,2019.0,123800.0,0.0,0.0,9600.0,1930.0,0.0,0.0,2019.0,7.0,12.0,1.0,0.0,1.0,0.0,11.0,15.0,0.0,2.0,7.0,6.0,4.0,1.0,0.0,0.0,2296.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2019.0,178812.509587,205000.0,-26187.490413
53,10434,2019.0,123800.0,0.0,0.0,9600.0,1930.0,0.0,0.0,2019.0,7.0,12.0,1.0,0.0,1.0,0.0,11.0,15.0,0.0,2.0,7.0,6.0,4.0,1.0,0.0,0.0,2296.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2019.0,176269.606922,205000.0,-28730.393078
52,10434,2019.0,123800.0,0.0,0.0,9600.0,1930.0,0.0,0.0,2019.0,7.0,12.0,1.0,0.0,1.0,0.0,11.0,15.0,0.0,2.0,7.0,6.0,4.0,1.0,0.0,0.0,2296.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2019.0,176999.321548,205000.0,-28000.678452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,4212,2019.0,93400.0,0.0,0.0,3200.0,1998.0,0.0,0.0,2019.0,2.0,13.0,2.0,3.0,4.0,0.0,10.0,3.0,0.0,2.0,6.0,6.0,4.0,1.0,0.0,0.0,932.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,124721.448679,90700.0,34021.448679
38,4212,2019.0,93400.0,0.0,0.0,3200.0,1998.0,0.0,0.0,2019.0,2.0,13.0,2.0,3.0,4.0,0.0,10.0,3.0,0.0,2.0,6.0,6.0,4.0,1.0,0.0,0.0,932.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2019.0,124172.109338,90700.0,33472.109338
39,4212,2019.0,93400.0,0.0,0.0,3200.0,1998.0,0.0,0.0,2019.0,2.0,13.0,2.0,3.0,4.0,0.0,10.0,3.0,0.0,2.0,6.0,6.0,4.0,1.0,0.0,0.0,932.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2019.0,122966.307017,90700.0,32266.307017
40,4212,2019.0,93400.0,0.0,0.0,3200.0,1998.0,0.0,0.0,2019.0,2.0,13.0,2.0,3.0,4.0,0.0,10.0,3.0,0.0,2.0,6.0,6.0,4.0,1.0,0.0,0.0,932.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2019.0,127277.894869,90700.0,36577.894869


In [265]:
#Checking y_delta/y_percent for each house saleID and change made to features
(df_houseBoth[["Prop_Addr"]]
    .join(df_predict.set_index("SaleID"), how = "right")
    .eval("y_percent = (y_delta/y_true)*100")
    .sort_values("y_true", ascending = False)
    [["YrBuilt", "HouseStyle", "BldgType", "y_delta", "y_percent"]]
)

,YrBuilt,HouseStyle,BldgType,y_delta,y_percent
SaleID,,,,,
10434,1930.0,0.0,1.0,-26187.490413,-12.774386
10434,1930.0,0.0,1.0,-10415.662182,-5.080811
10434,1930.0,0.0,1.0,-28730.393078,-14.014826
10434,1930.0,0.0,1.0,-28000.678452,-13.658868
10434,1930.0,0.0,1.0,-26976.013330,-13.159031
...,...,...,...,...,...
4212,1998.0,3.0,2.0,36577.894869,40.328440
4212,1998.0,3.0,2.0,32266.307017,35.574760
4212,1998.0,3.0,2.0,33472.109338,36.904200


In [298]:
#Selected SaleIDs 14817, 22472, and 4212 for Sample House Sale section of presentation, displaying their features below
df_feature_allBoth.query("index == 14817 or index == 22472 or index == 4212")

,Rcrd_Yr,InstOne_Yr,TtlVal_AsrYr,LotArea,YrBuilt,TtlBsmtSF,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,Has_Fireplace,Has_Garage,Has_Bsmt,TotalArea,TotalArea_Gar,AsbShng_Ext,AsphShn_Ext,BrkComm_Ext,BrkFace_Ext,CBlock_Ext,CemntBd_Ext,HdBoard_Ext,ImStucc_Ext,MetalSd_Ext,No_Data_Ext,Other_Ext,Plywood_Ext,Stucco_Ext,VinylSd_Ext,Wd_Sdng_Ext,WdShing_Ext,Two_Types_GrType,Attachd_GrType,Basment_GrType,BuiltIn_GrType,Carport_GrType,Detachd_GrType,No_Data_GrType,None_GrType,One_Story_HsStyle,One_Five_Fin_HsStyle,One_Five_Unf_HsStyle,Two_Story_HsStyle,Two_Five_Fin_HsStyle,Two_Five_Unf_HsStyle,No_Data_HsStyle,S_Foyer_HsStyle,S_Level_HsStyle,Closest_Time,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Gym_closest,Service_High_School_closest,Service_Historic_closest,Service_ISU_closest,Service_Library_closest,Service_Medical_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Closest_5,Convenience_secs,Central_historic_feature,biz_180_secs,biz_480_secs,Year,Stone_Ext
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4212,2019.0,2019.0,109900,3200.0,1998.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,932.0,932.0,0,0.0,0,1,0.0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,285.5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,370.20,492.0,548.6,0,8,2019,0.0
14817,2019.0,2019.0,113800,8050.0,1947.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1137.0,1137.0,0,0.0,0,0,0.0,0,0,0,2,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,240.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,268.88,436.9,520.8,0,9,2019,0.0
22472,2021.0,2021.0,99400,1472.0,1970.0,546.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1638.0,1638.0,0,0.0,0,0,0.0,2,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,294.5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,313.18,420.3,409.3,0,14,2019,0.0


## Constructing Final Data Table for Output to Presentation

In [266]:
df_display = (df_predict
    .query("SaleID == 4212 or SaleID == 22472 or SaleID == 14817")
    [["SaleID", 
    "YrBuilt", 
    "Has_Fireplace", 
    "Has_Garage", 
    "Service_Organic_Groceries_closest", 
    "Service_Restaurant_closest", 
    "Service_Shopping_closest", 
    "Service_Religion_closest", 
    "Year", 
    "y_true", 
    "y_predict", 
    "y_delta"]]
    .eval("y_percent = (y_delta/y_true)*100")
)


df_display2 = (df_display.filter(regex = "Has_|_closest|Year")
    .stack()
    .rename("flag")
    .reset_index()
    .query("flag == 1 or flag == 2021.0")
    .set_index("level_0")
    .drop("flag", axis = 1)
    .apply(lambda x: x.str.replace("Has_", "Add "))
    .apply(lambda x: x.str.replace("_closest", ""))
    .apply(lambda x: x.str.replace("Service_", "Closest to "))
    .apply(lambda x: x.str.replace("_", " "))
    .apply(lambda x: x.str.replace("Year", "Year Sold 2021"))
    ["level_1"]
    .rename("Changes")
    .to_frame()
    .join(df_display)
    .filter(regex = "Changes|SaleID|YrBuilt|Year|y_true|y_predict|y_perc")
    .assign(YrBuilt = lambda x: x.YrBuilt.astype(int))
    .assign(y_predict = lambda x: "$" + x.y_predict.astype(int).astype(str))
    .assign(y_true = lambda x: "$" + x.y_true.astype(int).astype(str))
    .assign(y_percent = lambda x: x.y_percent.round(2).astype(str) + "%")
)

display_dict = {}

for i in df_display2.SaleID.unique():
    k = df_display2.query("SaleID == @i").head(1).apply(lambda x: f"Year Built:{x.YrBuilt}, SaleID:{x.SaleID}, Original: {x.y_true}", axis = 1).values[0]
    index = df_display2.query("SaleID == @i").Changes.reset_index(drop = True)
    display_dict[(k,"Prediction")] = df_display2.query("SaleID == @i").set_index(index).y_predict
    display_dict[(k,"Difference")] = df_display2.query("SaleID == @i").set_index(index).y_percent

pd.DataFrame(display_dict)#.to_csv("Visualizations/df_predictions.csv")

Year Built:1970, SaleID:22472, Original: $101000  \
                                                                   Prediction   
Changes                                                                         
Add Fireplace                                                         $124944   
Add Garage                                                            $113701   
Closest to Organic Groceries                                          $115798   
Closest to Restaurant                                                 $114430   
Closest to Shopping                                                   $112033   
Closest to Religion                                                   $111464   
Year Sold 2021                                                        $121251   

                                         \
                             Difference   
Changes                                   
Add Fireplace                    23.71%   
Add Garage                       12.58%   
Closest to Organic Groceries     14.65%   
Closest to Restaurant             13.3%   
Closest to Shopping              10.92%   
Closest to Religion              10.36%   
Year Sold 2021                   20.05%   

                             Year Built:1998, SaleID:4212, Original: $90700  \
                                                                 Prediction   
Changes                                                                       
Add Fireplace                                                       $126428   
Add Garage                                                          $124172   
Closest to Organic Groceries                                        $124721   
Closest to Restaurant                                               $124172   
Closest to Shopping                                                 $122966   
Closest to Religion                                                 $127277   
Year Sold 2021                                                      $130130   

                                         \
                             Difference   
Changes                                   
Add Fireplace                    39.39%   
Add Garage                        36.9%   
Closest to Organic Groceries     37.51%   
Closest to Restaurant             36.9%   
Closest to Shopping              35.57%   
Closest to Religion              40.33%   
Year Sold 2021                   43.47%   

                             Year Built:1947, SaleID:14817, Original: $134000  \
                                                                   Prediction   
Changes                                                                         
Add Fireplace                                                         $148369   
Add Garage                                                            $138646   
Closest to Organic Groceries                                          $140153   
Closest to Restaurant                                                 $139017   
Closest to Shopping                                                   $136978   
Closest to Religion                                                   $141751   
Year Sold 2021                                                        $142253   

                                         
                             Difference  
Changes                                  
Add Fireplace                    10.72%  
Add Garage                        3.47%  
Closest to Organic Groceries      4.59%  
Closest to Restaurant             3.74%  
Closest to Shopping               2.22%  
Closest to Religion               5.78%  
Year Sold 2021                    6.16%